## ***Initializing and Importing Packages***

In [1]:
using Plots, DataFrames, StatsBase, Plots.PlotMeasures,
    LaTeXStrings, CSV, ProgressMeter, Dates, JLD, JSON, HTTP

DataDir = "../../Data/"
FigsDir = "../../Figs/"

include("Modules.jl")

# theme(:dark)
Plots.default(titlefontsize=12, tickfontsize=10, labelfontsize=12, legendfontsize=9,
    rightmargin=5mm, bottommargin=5mm, topmargin=5mm, leftmargin=5mm,
    fontfamily="Computer Modern", frame=:box, label=nothing)

## ***BTC Moving Correlation Analysis***

In [2]:
BTCFIATDataFrames = load(DataDir * "Raw-Data/CoinGecko/Max-Data-Daily/Chart-Data/BTC-FIAT-DataFrames.jld");
Markets = collect(keys(BTCFIATDataFrames));

In [110]:
StartPoint, EndPoint, SpanLen = 500, 3400, 60
dates = BTCFIATDataFrames["USD"].Date
BTCPriceCorMatrices = [cor(hcat([BTCFIATDataFrames[market].Price[i:i+SpanLen]
                         for market ∈ Markets]...)) for i ∈ StartPoint:EndPoint-SpanLen];
BTCVolumeCorMatrices = [cor(hcat([BTCFIATDataFrames[market].TotalVolume[i:i+SpanLen]
                                  for market ∈ Markets]...)) for i ∈ StartPoint:EndPoint-SpanLen];
MarketsMovingBTCPriceCor = Dict()
MarketsMovingBTCVolumeCor = Dict()
for i ∈ 1:length(Markets)
    marketPricecordata = Dict()
    marketVolumecordata = Dict()
    Pricemarket = [(sum(CorMat[i, :]) - 1) / (size(CorMat)[1] - 1) for CorMat ∈ BTCPriceCorMatrices]
    Volumemarket = [(sum(CorMat[i, :]) - 1) / (size(CorMat)[1] - 1) for CorMat ∈ BTCVolumeCorMatrices]
    date = Date.(dates[StartPoint:EndPoint-SpanLen])
    marketPricecordata["MovingCor"] = Pricemarket
    marketVolumecordata["MovingCor"] = Volumemarket
    marketPricecordata["Date"] = date
    marketVolumecordata["Date"] = date
    MarketsMovingBTCPriceCor[Markets[i]] = marketPricecordata
    MarketsMovingBTCVolumeCor[Markets[i]] = marketVolumecordata
end

SelectedMarkets = ["USD", "EUR", "CNY", "RUB", "GBP", "UAH", "NGN", "ARS", "BRL", "TRY"];

In [181]:
spoint = 2600
# epoint = 2500
Markers = [:o, :square, :diamond, :hexagon, :cross, :plus]
dates = Date.(MarketsMovingBTCPriceCor["USD"]["Date"][spoint:end])
plot(ylabel="Moving Correlation", size=(1100, 600))
plot!(dates, MarketsMovingBTCPriceCor["RUB"]["MovingCor"][spoint:end], label="RUB", m=Markers[5], msw=1, palette=palette(:romaO, 6))
selmarks = ["USD", "EUR", "CNY", "GBP"]
for i ∈ 1:length(selmarks)
    Market = selmarks[i]
    plot!(dates[1:10:end], MarketsMovingBTCPriceCor[Market]["MovingCor"][spoint:10:end],
        label=Market, m=Markers[i], palette=palette(:romaO, 6), ms=3, msw=0.5)
end
Plot_1 = plot!(xticks=dates[1:60:length(dates)], xrotation=15, legend=:bottomleft)
savefig(FigsDir * "MovingCorrelation-BTCPrice-FinnoRussoWar.pdf")
plot!(dpi=300)
savefig(FigsDir * "MovingCorrelation-BTCPrice-FinnoRussoWar.png")

In [180]:
spoint = 2600
# epoint = 2500
Markers = [:o, :square, :diamond, :hexagon, :cross, :plus]
dates = Date.(MarketsMovingBTCVolumeCor["USD"]["Date"][spoint:end])
plot(ylabel="Moving Correlation", size=(1100, 600))
plot!(dates, MarketsMovingBTCVolumeCor["RUB"]["MovingCor"][spoint:end], label="RUB", m=Markers[5], msw=1, palette=palette(:romaO, 6))
selmarks = ["USD", "EUR", "CNY", "GBP"]
for i ∈ 1:length(selmarks)
    Market = selmarks[i]
    plot!(dates[1:10:end], MarketsMovingBTCVolumeCor[Market]["MovingCor"][spoint:10:end],
        label=Market, m=Markers[i], palette=palette(:romaO, 6), ms=3, msw=0.5)
end
Plot_2 = plot!(xticks=dates[1:60:length(dates)], xrotation=15, legend=:bottomleft)
savefig(FigsDir * "MovingCorrelation-BTCVolume-FinnoRussoWar.pdf")
plot!(dpi=300)
savefig(FigsDir * "MovingCorrelation-BTCVolume-FinnoRussoWar.png")

In [179]:
spoint = 1
epoint = 2600

Markers = [:o, :square, :diamond, :hexagon, :s, :p, :dtriangle, :utriangle,
    :pentagon, :star6, :hexagon3, :star4];

dates = Date.(MarketsMovingBTCPriceCor["USD"]["Date"][spoint:end])
plot(ylabel="Moving Correlation", size=(1100, 600))
for i ∈ 1:length(SelectedMarkets)
    Market = SelectedMarkets[i]
    plot!(dates, MarketsMovingBTCPriceCor[Market]["MovingCor"][spoint:end],
        label=Market, m=Markers[i], ms=2, msw=0, palette=palette(:romaO, 11))
end
Plot_3 = plot!(xticks=dates[1:200:length(dates)], xrotation=15, legend=:bottomleft)
savefig(FigsDir * "MovingCorrelation-BTCPrice-Overview.pdf")
plot!(dpi=300)
savefig(FigsDir * "MovingCorrelation-BTCPrice-Overview.png")

In [177]:
spoint = 1
epoint = 2600

Markers = [:o, :square, :diamond, :hexagon, :s, :p, :dtriangle, :utriangle,
    :pentagon, :star6, :hexagon3, :star4];

dates = Date.(MarketsMovingBTCVolumeCor["USD"]["Date"][spoint:end])
plot(ylabel="Moving Correlation", size=(1100, 600))
for i ∈ 1:length(SelectedMarkets)
    Market = SelectedMarkets[i]
    plot!(dates, MarketsMovingBTCVolumeCor[Market]["MovingCor"][spoint:end],
        label=Market, m=Markers[i], ms=2, msw=0, palette=palette(:romaO, 11))
end
Plot_4 = plot!(xticks=dates[1:200:length(dates)], xrotation=15, legend=:bottomleft)
savefig(FigsDir * "MovingCorrelation-BTCVolume-Overview.pdf")
plot!(dpi=300)
savefig(FigsDir * "MovingCorrelation-BTCVolume-Overview.png")

## ***ETH Moving Correlation Analysis***

In [114]:
ETHFIATDataFrames = load(DataDir * "Raw-Data/CoinGecko/Max-Data-Daily/Chart-Data/ETH-FIAT-DataFrames.jld");
Markets = collect(keys(ETHFIATDataFrames));

In [162]:
StartPoint, EndPoint, SpanLen = 1, 2570, 60
dates = ETHFIATDataFrames["USD"].Date
ETHPriceCorMatrices = [cor(hcat([ETHFIATDataFrames[market].Price[i:i+SpanLen]
                         for market ∈ Markets]...)) for i ∈ StartPoint:EndPoint-SpanLen];
ETHVolumeCorMatrices = [cor(hcat([ETHFIATDataFrames[market].TotalVolume[i:i+SpanLen]
                                  for market ∈ Markets]...)) for i ∈ StartPoint:EndPoint-SpanLen];
MarketsMovingETHPriceCor = Dict()
MarketsMovingETHVolumeCor = Dict()
for i ∈ 1:length(Markets)
    marketPricecordata = Dict()
    marketVolumecordata = Dict()
    Pricemarket = [(sum(CorMat[i, :]) - 1) / (size(CorMat)[1] - 1) for CorMat ∈ ETHPriceCorMatrices]
    Volumemarket = [(sum(CorMat[i, :]) - 1) / (size(CorMat)[1] - 1) for CorMat ∈ ETHVolumeCorMatrices]
    date = Date.(dates[StartPoint:EndPoint-SpanLen])
    marketPricecordata["MovingCor"] = Pricemarket
    marketVolumecordata["MovingCor"] = Volumemarket
    marketPricecordata["Date"] = date
    marketVolumecordata["Date"] = date
    MarketsMovingETHPriceCor[Markets[i]] = marketPricecordata
    MarketsMovingETHVolumeCor[Markets[i]] = marketVolumecordata
end

In [163]:
spoint = 2300
# epoint = 2500
Markers = [:o, :square, :diamond, :hexagon, :cross, :plus]
dates = Date.(MarketsMovingETHPriceCor["USD"]["Date"][spoint:end])
plot(ylabel="Moving Correlation", size=(1100, 600))
plot!(dates, MarketsMovingETHPriceCor["RUB"]["MovingCor"][spoint:end], label="RUB", m=Markers[5], msw=1, palette=palette(:romaO, 6))
selmarks = ["USD", "EUR", "CNY", "GBP"]
for i ∈ 1:length(selmarks)
    Market = selmarks[i]
    plot!(dates[1:10:end], MarketsMovingETHPriceCor[Market]["MovingCor"][spoint:10:end],
        label=Market, m=Markers[i], palette=palette(:romaO, 6), ms=3, msw=0.5)
end
plot!(xticks=dates[1:60:length(dates)], xrotation=15, legend=:bottomleft)
savefig(FigsDir * "MovingCorrelation-ETHPrice-FinnoRussoWar.pdf")
plot!(dpi=300)
savefig(FigsDir * "MovingCorrelation-ETHPrice-FinnoRussoWar.png")

In [164]:
spoint = 2300
# epoint = 2500
Markers = [:o, :square, :diamond, :hexagon, :cross, :plus]
dates = Date.(MarketsMovingETHVolumeCor["USD"]["Date"][spoint:end])
plot(ylabel="Moving Correlation", size=(1100, 600))
plot!(dates, MarketsMovingETHVolumeCor["RUB"]["MovingCor"][spoint:end], label="RUB", m=Markers[5], msw=1, palette=palette(:romaO, 6))
selmarks = ["USD", "EUR", "CNY", "GBP"]
for i ∈ 1:length(selmarks)
    Market = selmarks[i]
    plot!(dates[1:10:end], MarketsMovingETHVolumeCor[Market]["MovingCor"][spoint:10:end],
        label=Market, m=Markers[i], palette=palette(:romaO, 6), ms=3, msw=0.5)
end
plot!(xticks=dates[1:60:length(dates)], xrotation=15, legend=:bottomleft)
savefig(FigsDir * "MovingCorrelation-ETHVolume-FinnoRussoWar.pdf")
plot!(dpi=300)
savefig(FigsDir * "MovingCorrelation-ETHVolume-FinnoRussoWar.png")

In [165]:
spoint = 1
epoint = 2300

Markers = [:o, :square, :diamond, :hexagon, :s, :p, :dtriangle, :utriangle,
    :pentagon, :star6, :hexagon3, :star4];

dates = Date.(MarketsMovingETHPriceCor["USD"]["Date"][spoint:end])
plot(ylabel="Moving Correlation", size=(1100, 600))
for i ∈ 1:length(SelectedMarkets)
    Market = SelectedMarkets[i]
    plot!(dates, MarketsMovingETHPriceCor[Market]["MovingCor"][spoint:end],
        label=Market, m=Markers[i], ms=2, msw=0, palette=palette(:romaO, 11))
end
plot!(xticks=dates[1:200:length(dates)], xrotation=15, legend=:bottomleft)
savefig(FigsDir * "MovingCorrelation-ETHPrice-Overview.pdf")
plot!(dpi=300)
savefig(FigsDir * "MovingCorrelation-ETHPrice-Overview.png")

In [167]:
spoint = 1
epoint = 2300

Markers = [:o, :square, :diamond, :hexagon, :s, :p, :dtriangle, :utriangle,
    :pentagon, :star6, :hexagon3, :star4];

dates = Date.(MarketsMovingETHVolumeCor["USD"]["Date"][spoint:end])
plot(ylabel="Moving Correlation", size=(1100, 600))
for i ∈ 1:length(SelectedMarkets)
    Market = SelectedMarkets[i]
    plot!(dates, MarketsMovingETHVolumeCor[Market]["MovingCor"][spoint:end],
        label=Market, m=Markers[i], ms=2, msw=0, palette=palette(:romaO, 11))
end
plot!(xticks=dates[1:200:length(dates)], xrotation=15, legend=:bottomleft)
savefig(FigsDir * "MovingCorrelation-ETHVolume-Overview.pdf")
plot!(dpi=300)
savefig(FigsDir * "MovingCorrelation-ETHVolume-Overview.png")

## ***Crypro-FIAT Analysis***

In [117]:
# FIATDataFrames = load(DataDir * "Raw-Data/CoinGecko/Max-Data-Daily/Chart-Data/FIAT-Crypto-DataFrames.jld");
# FIAT1HourDataFrames = load(DataDir * "Raw-Data/CoinGecko/Max-Data-Intraday/FIAT-Crypto-1Hour-DataFrames.jld");
CryptoFiatCorData = load(DataDir * "Raw-Data/CoinGecko/Max-Data-Daily/Chart-Data/Fiat-Crypto-Splited.jld")

In [118]:
Markets = String["USD", "CNY", "EUR", "JPY", "GBP",
    "KRW", "INR", "CAD", "HKD", "AUD", "BRL", "CHF",
    "RUB", "MXN", "SGD", "IDR", "AED", "MYR", "PLN",
    "TRY", "NZD", "ZAR", "CZK", "ARS", "DKK", "NOK",
    "SEK", "NGN", "BHD", "LKR", "UAH"];
Symbols = String["ADA", "BCH", "BNB", "BTC", "DASH", "DOGE", "EOS",
    "ETC", "ETH", "LSK", "LTC", "NEO", "OMG",
    "TRX", "XEM", "XLM", "XMR", "XRP"];

SelectedMarkets = ["USD", "EUR", "CNY", "RUB",
    "GBP", "UAH", "NGN", "ARS", "BRL", "TRY"];

In [119]:
# CryptoFiatCorData = Dict()
# for Crypto ∈ Symbols
#     MarketsData = Dict()
#     for Market ∈ Markets
#         MarketsData[Market] = dropmissing(FIATDataFrames["$Crypto-$Market"])
#     end
#     CryptoFiatCorData[Crypto] = MarketsData
# end

### ***Moving Correlation***

In [120]:
SpanLen = 40
CryptoMarketsData = Dict()
p = Progress(length(Symbols))
for Crypto ∈ Symbols
    next!(p)
    TotalDates = intersect([CryptoFiatCorData[Crypto][Market].Date for Market in Markets]...)
    TotalDataFrames = [filter(row -> row.Date ∈ TotalDates, CryptoFiatCorData[Crypto][Market]) for Market in Markets]
    TotalPrices = hcat([df.Price for df ∈ TotalDataFrames]...)
    TotalVolumes = hcat([df.TotalVolume for df ∈ TotalDataFrames]...)
    PriceCorMatrices = Matrix[]
    VolumeCorMatrices = Matrix[]
    for i ∈ 1:length(TotalDates)-SpanLen
        PriceMatrix = TotalPrices[i:i+SpanLen, :]
        TotalVolumeMatrix = TotalVolumes[i:i+SpanLen, :]
        CorMatPrice = cor(PriceMatrix)
        CorMatTotalVolume = cor(TotalVolumeMatrix)
        push!(PriceCorMatrices, CorMatPrice)
        push!(VolumeCorMatrices, CorMatTotalVolume)
    end

    ################################################################
    ################################################################
    
    MarketsCorMatrices = Dict()
    for i ∈ 1:length(Markets)
        Pricemarket = [(sum(CorMat[i, :]) - 1) / (size(CorMat)[1] - 1) for CorMat ∈ PriceCorMatrices]
        Volumemarket = [(sum(CorMat[i, :]) - 1) / (size(CorMat)[1] - 1) for CorMat ∈ VolumeCorMatrices]
        MarketsCorMatrices[Markets[i]] = Dict("Date" => Date.(TotalDates[1:end-SpanLen]), "PriceCorrelation" => Pricemarket, "VolumeCorrelation" => Volumemarket)
    end
    CryptoMarketsData[Crypto] = MarketsCorMatrices
end

# save(DataDir * "Processed-Data/FIAT-Crypto-Daily-MovCor.jld", CryptoMarketsData);

Progress: 100%|█████████████████████████████████████████| Time: 0:00:21


In [121]:
PLTs = Dict()
for Crypto ∈ Symbols
    PricePLT = begin
        plot(title="Moving Correlation of Prices for top Markets ($Crypto)",
            xlabel="Date", ylabel="Moving Correlation", size=(1100, 600))
        for Market ∈ SelectedMarkets
            plot!(CryptoMarketsData[Crypto][Market]["Date"], CryptoMarketsData[Crypto][Market]["PriceCorrelation"], label=Market)
        end
        len = length(CryptoMarketsData[Crypto]["USD"]["Date"])
        plot!(xticks=CryptoMarketsData[Crypto]["USD"]["Date"][1:round(Int, len / 25):len], xrotation=30, legend=nothing)
    end
    VolumePLT = begin
        plot(title="Moving Correlation of Volume3 for top Markets ($Crypto)",
            xlabel="Date", ylabel="Moving Correlation", size=(1100, 600))
        for Market ∈ SelectedMarkets
            plot!(CryptoMarketsData[Crypto][Market]["Date"], CryptoMarketsData[Crypto][Market]["VolumeCorrelation"], label=Market)
        end
        len = length(CryptoMarketsData[Crypto]["USD"]["Date"])
        plot!(xticks=CryptoMarketsData[Crypto]["USD"]["Date"][1:round(Int, len / 25):len], xrotation=30, legend=nothing)
    end
    PLTs[Crypto] = Dict("Price" => PricePLT, "Volume" => VolumePLT)
    # savefig(FigsDir * "FIAT-Crypto-Daily-Price-Correlation-" * Crypto * ".pdf")
end

In [ ]:
PLTs["XRP"]["Price"]

### ***Volatility***

In [176]:
FIATForExData = Dict()

for Market ∈ Markets
    FIATForExData[Market] = Dict()
    for ToMarket ∈ Markets
        DataLoadDir = DataDir * "Raw-Data/AlphaVantage/ForEx-Daily/$Market-$ToMarket.csv"
        if isfile(DataLoadDir)
            FIATForExData[Market][ToMarket] = CSV.read(DataLoadDir, DataFrame)
        end
    end
end

In [193]:
SinkedToUSDData = Dict()
for Crypto ∈ Symbols
    SinkedToUSDData[Crypto] = Dict()
    for Market ∈ Markets
        ComTimes = intersect(Date.(CryptoFiatCorData[Crypto][Market].Date), FIATForExData[Market]["USD"].timestamp)
        df1 = filter(row -> row.Date ∈ ComTimes, CryptoFiatCorData[Crypto][Market])
        df2 = sort(filter(row -> row.timestamp ∈ ComTimes && row.timestamp ∈ df1.Date, FIATForExData[Market]["USD"]), :timestamp)
        forexPrice = (df2.high .+ df2.low) / 2
        cryproprice = df1.Price
        SinkedToUSDData[Crypto][Market] = Dict("Date" => df1.Date, "Price" => cryproprice .* forexPrice)
    end
end

In [223]:
plot(ylab = "Moving Volatility")
for Market ∈ SelectedMarkets
    data = ExtraStats.CalculateMovingVolatility(SinkedToUSDData["BTC"][Market], 40)
    plot!(data["Date"], data["MovingVolatility"], label=Market)
end
plot!(size = (1100, 600))

: 

In [199]:
plot(SinkedToUSDData["BTC"]["USD"]["Date"], SinkedToUSDData["BTC"]["USD"]["Price"], title="BTC Price in USD", xlabel="Date", ylabel="Price in USD", label="USD")
plot!(SinkedToUSDData["BTC"]["GBP"]["Date"], SinkedToUSDData["BTC"]["GBP"]["Price"], label="GBP")
plot!(SinkedToUSDData["BTC"]["EUR"]["Date"], SinkedToUSDData["BTC"]["EUR"]["Price"], label="EUR")
plot!(SinkedToUSDData["BTC"]["CNY"]["Date"], SinkedToUSDData["BTC"]["CNY"]["Price"], label="CNY")
plot!(SinkedToUSDData["BTC"]["RUB"]["Date"], SinkedToUSDData["BTC"]["RUB"]["Price"], label="RUB")
plot!(size = (1100, 600))